In [ ]:
!pip install datasets

from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
# Download NLTK stop words
nltk.download('stopwords')
nltk.download('punkt')

#imports
from transformers import AutoTokenizer, TFAutoModel

!pip install -q transformers==4.37.2

from transformers import BertTokenizer

!pip install pydot

# Load the dataset
dataset = load_dataset("orieg/elsevier-oa-cc-by")
elsevier_dataset = dataset

train_set = elsevier_dataset['train']
train_df = pd.DataFrame(train_set)

val_set = elsevier_dataset['validation']
val_df = pd.DataFrame(val_set)

test_set = elsevier_dataset['test']
test_df = pd.DataFrame(test_set)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 28.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for orieg/elsevier-oa-cc-by contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/orieg/elsevier-oa-cc-by
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datas

Generating train split:   0%|          | 0/32072 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4009 [00:00<?, ? examples/s]

In [ ]:
def prepare_single_label_data(train_df, val_df, test_df):
    """
    Prepares the Elsevier corpus splits focusing on single-label classification.
    """
    # Filter each DataFrame to include only single-label instances
    single_label_train_df = train_df[train_df['subjareas'].apply(len) == 1]
    single_label_val_df = val_df[val_df['subjareas'].apply(len) == 1]
    single_label_test_df = test_df[test_df['subjareas'].apply(len) == 1]

    # Initialize the MultiLabelBinarizer
    mlb = MultiLabelBinarizer()

    all_labels = pd.concat([
        single_label_train_df['subjareas'],
        single_label_val_df['subjareas'],
        single_label_test_df['subjareas']
    ])
    mlb.fit(all_labels)

    train_labels = mlb.transform(single_label_train_df['subjareas'])
    val_labels = mlb.transform(single_label_val_df['subjareas'])
    test_labels = mlb.transform(single_label_test_df['subjareas'])

    train_texts = single_label_train_df['abstract'].tolist()
    val_texts = single_label_val_df['abstract'].tolist()
    test_texts = single_label_test_df['abstract'].tolist()

    return train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, mlb.classes_

train_texts, val_texts, test_texts, train_labels, val_labels, test_labels, target_names = prepare_single_label_data(train_df, val_df, test_df)

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

# Model checkpoint
model_checkpoint = 'allenai/scibert_scivocab_cased'

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load the model from PyTorch weights
model = TFAutoModel.from_pretrained(model_checkpoint, from_pt=True)

max_length = 300

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
valid_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length, return_tensors='tf')

def create_scibert_singlelabel_model(model, num_classes, max_length, hidden_size=201, dropout=0.3, learning_rate=0.00005):
    """
    Build a classification model with SciBERT for single-label classification.
    """
    model.trainable = True

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask')

    bert_output = model(input_ids=input_ids, attention_mask=attention_mask)
    pooled_output = bert_output.pooler_output

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(pooled_output)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)
    output_classification = tf.keras.layers.Dense(num_classes, activation='softmax')(hidden)  # Changed to softmax for single-label classification

    classification_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_classification)
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),  # Changed to CategoricalCrossentropy
                                 metrics=['accuracy'])

    return classification_model

num_subject_areas = len(target_names)

# Creating the model specifically for single-label classification
scibert_model = create_scibert_singlelabel_model(model=model,
                                                 num_classes=num_subject_areas,
                                                 max_length=max_length,
                                                 hidden_size=201,
                                                 dropout=0.3,
                                                 learning_rate=0.00005)

scibert_model.summary()

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 300)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 300)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1099384   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   32         'attention_mask[0][0]']      
                             hidden_state=(None, 300, 7                                       

In [ ]:
np_train_labels = np.asarray(train_labels)
np_valid_labels = np.asarray(val_labels)

# Training the model for single-label classification
scibert_model_history = scibert_model.fit(
    [train_encodings.input_ids, train_encodings.attention_mask],
    np_train_labels,
    validation_data=(
        [valid_encodings.input_ids, valid_encodings.attention_mask],
        np_valid_labels
    ),
    batch_size=8,
    epochs=1
)

1792/1792 [==============================] - 413s 206ms/step - loss: 1.1586 - accuracy: 0.6659 - val_loss: 1.0781 - val_accuracy: 0.6839


In [ ]:
np_test_labels = np.asarray(test_labels)

# Evaluating the model on the test dataset
test_score = scibert_model.evaluate(
    [test_encodings.input_ids, test_encodings.attention_mask],
    np_test_labels
)

print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

55/55 [==============================] - 12s 220ms/step - loss: 1.0486 - accuracy: 0.6930
Test loss: 1.0485658645629883
Test accuracy: 0.692967414855957


In [ ]:
# Making predictions on the test dataset
predictions = scibert_model.predict(
    [test_encodings.input_ids, test_encodings.attention_mask]
)

predicted_class_indices = np.argmax(predictions, axis=1)

predicted_class_names = [target_names[class_index] for class_index in predicted_class_indices]
print(predicted_class_names[:5])


55/55 [==============================] - 16s 216ms/step
['MEDI', 'SOCI', 'PHYS', 'MULT', 'MULT']


In [ ]:
from sklearn.metrics import classification_report

all_class_indices = range(num_subject_areas)

report = classification_report(true_class_indices, predicted_class_indices, labels=all_class_indices, target_names=target_names)

print(report)


              precision    recall  f1-score   support

        AGRI       0.80      0.72      0.75       144
        ARTS       0.00      0.00      0.00         3
        BIOC       0.80      0.74      0.77       190
        BUSI       1.00      0.28      0.43        18
        CENG       0.00      0.00      0.00         9
        CHEM       0.50      0.07      0.12        14
        COMP       0.90      0.63      0.75        30
        DECI       0.00      0.00      0.00         0
        DENT       0.00      0.00      0.00         0
        EART       0.91      0.68      0.78        91
        ECON       0.54      0.65      0.59        20
        ENER       0.67      0.55      0.60        53
        ENGI       0.83      0.23      0.36        22
        ENVI       0.74      0.41      0.53       135
        HEAL       0.00      0.00      0.00         1
        IMMU       0.65      0.46      0.54        24
        MATE       0.93      0.24      0.38        54
        MATH       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.